In [36]:
import requests
import sqlite3
from lxml import etree
import settings
import datetime

In [37]:
class utils(object):
    def __init__(self, db_location=settings.db_location):
        self.db_location = db_location
        
    def _connect_to_db(self):
        """Returns connection to package sqlite db, which holds all source urls 
        as well as articles which have been previously uplaoded to Instapaper by the application"""
        con = sqlite3.connect(self.db_location)
        return con
    

In [38]:
class inbound_local(utils):
    def __init__(self, db_location=settings.db_location):
        self.db_location = db_location
    
    def _gather_sources(self):
        """Gathers active sources from the package db which will be referenced
        by the application when seeking newly published articles"""
        source_query = """SELECT name, connect_point, type FROM sources WHERE status=1;"""
        con = self._connect_to_db()
        c = con.cursor()
        
        source_dict = {}
        for row in c.execute(source_query):
            name, connect_point, _type = row
            if _type in source_dict.keys():
                source_dict[_type].append((name,connect_point))
            else:
                source_dict[_type] = [(name,connect_point)]
        con.commit()
        con.close()
        return source_dict

    def _rss_gather(self, source_name, rss_feed_link):
        """Pull all articles given a rss feed link
        Details that are passed back will mirror those required for insertion into the articles table
        (name, source, connect_point, publish_time)"""
        tree = etree.fromstring(requests.get(rss_feed_link).text)
        articles = tree.findall('.//item')

        articles_details = []
        status = 0
        _type = 'RSS'
        for article in articles:
            articles_details.append((article.find('title').text, source_name,
                                     article.find('link').text, article.find('pubDate').text))
        return articles_details

    def _store_article_details(self):
        """First checks that there are articles waiting to be written into the local db
        then connects and inserts each article into the `articles` table"""
        
        assert len(self.collected_articles) > 0, "No articles collected."
        
        con = self._connect_to_db()
        c = con.cursor()
        insert_query = """INSERT INTO articles (name, source, connect_point, publish_time, insert_time, uploaded)
        VALUES(?,?,?,?,?,?)"""
        uploaded = False
        insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:00')
        
        article_insert = []
        for article in self.collected_articles:
            name, source, connect_point, publish_time = article
            article_insert.append((name, source, connect_point, publish_time, insert_time, uploaded))
        c.executemany(insert_query, article_insert)
        [print('|| {} ||'.format(' || '.join([str(x) for x in row]))) for row in article_insert]
        con.commit()
        con.close()
        return 0
                        
    def check_sources(self):
        source_dict = self._gather_sources()
        article_details = []
        if 'RSS' in source_dict.keys():
            for source in source_dict['RSS']:
                source_name, rss_feed_link = source
                article_details += self._rss_gather(source_name=source_name,
                                                        rss_feed_link=rss_feed_link)
                print('Sourced: {source_name}'.format(source_name=source_name))
        elif 'Medium' in source_dict.keys():
            for source in source_dict['Medium']:
                source_name, medium_link = source
                article_details += self._medium_gather(source_name=source_name,
                                                           medium_link=medium_link)
        self.collected_articles = article_details
        return 0
    
    def add_source(self, name, _type, connect_point, status):
        con = sqlite3.connect(self.db_location)
        c = con.cursor()
        insert_query = """INSERT INTO sources (name, type, connect_point, status, insert_time)
                VALUES(?,?,?,?,?)"""
        c.execute(
                      insert_query, (name, _type, connect_point, status, 
                      datetime.datetime.now().strftime('%Y-%m-%d %H:%M:00'))
                     )
        con.commit()
        con.close()
        return 0

    def _turn_off_source(self, source_name):
        con = sqlite3.connect(self.db_location)
        c = con.cursor()
        off_query = """UPDATE sources SET status=0 WHERE name=?"""
        c.execute(off_query, (source_name,))
        con.commit()
        con.close()
        print('{} turned off'.format(source_name))
        return 0

In [91]:
l_train = inbound_local()

In [92]:
l_train._turn_off_source(source_name='The Upshot')

The Upshot turned off


0

In [93]:
l_train.check_sources()

Sourced: Paul Krugman


0

In [94]:
con = sqlite3.connect(settings.db_location)
c = con.cursor()
for row in c.execute("SELECT * FROM sources;"):
    print('\t| '.join([str(x) for x in row]))
con.close()

Paul Krugman	| RSS	| http://www.nytimes.com/svc/collections/v1/publish/www.nytimes.com/column/paul-krugman/rss.xml	| 1	| 2019-10-24 16:39
The Upshot	| RSS	| https://rss.nytimes.com/services/xml/rss/nyt/Upshot.xml	| 0	| 2019-10-25 13:33:00


In [104]:
l_train._store_article_details()

|| Debt, Doomsayers and Double Standards || Paul Krugman || https://www.nytimes.com/2019/10/28/opinion/us-budget-deficit.html || Mon, 28 Oct 2019 22:30:08 GMT || 2019-10-30 17:05:00 || False ||
|| The Day the Trump Boom Died || Paul Krugman || https://www.nytimes.com/2019/10/24/opinion/trump-economy.html || Thu, 24 Oct 2019 20:35:56 GMT || 2019-10-30 17:05:00 || False ||
|| Can Warren Escape the Medicare Trap? || Paul Krugman || https://www.nytimes.com/2019/10/21/opinion/warren-medicare-for-all.html || Mon, 21 Oct 2019 22:30:05 GMT || 2019-10-30 17:05:00 || False ||
|| Democrats, Avoid the Robot Rabbit Hole || Paul Krugman || https://www.nytimes.com/2019/10/17/opinion/democrats-automation.html || Thu, 17 Oct 2019 21:53:37 GMT || 2019-10-30 17:05:00 || False ||
|| God Is Now Trump’s Co-Conspirator || Paul Krugman || https://www.nytimes.com/2019/10/14/opinion/trump-william-barr-speech.html || Mon, 14 Oct 2019 22:30:07 GMT || 2019-10-30 17:05:00 || False ||
|| Luckily, Trump Is an Unstabl

0

In [ ]:
class outbound_express(object):
    def __init__(self, db_location=settings.db_location):
        self.db_location = db_location
        self.instapaper_account_token = settings.credentials['instapaper']['API_TOKEN']
        
    def check_upload_needs(self, source_name):
        

In [102]:
con = sqlite3.connect(settings.db_location)
c = con.cursor()
c.execute("DELETE FROM articles;")
con.commit()